In [1]:
%cd /home/aditya/Documents/SolarCycleProject/code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import gaussian
import src.preprocesser as pre
import src.network as network
import src.plotter as plotter 
from src.hyperparams import *
%load_ext autoreload

/home/aditya/Documents/SolarCycleProject/code


Using TensorFlow backend.


In [13]:
%autoreload

In [27]:
# Getting the data
headers = ["Year",
           "Month",
           "Day",
           "Decimal Date",
           "Daily Total Sunspot Number",
           "Sunspot Number Stddev",
           "No of observations",
           "Definitive/Provisional"
]
filename = "data/SN_d_tot_V2.0.csv"
data = pd.read_csv(filename, delimiter=";", names=headers)[:10000]

In [28]:
data

,Year,Month,Day,Decimal Date,Daily Total Sunspot Number,Sunspot Number Stddev,No of observations,Definitive/Provisional
0,1818,1,1,1818.001,-1,-1.0,0,1
1,1818,1,2,1818.004,-1,-1.0,0,1
2,1818,1,3,1818.007,-1,-1.0,0,1
3,1818,1,4,1818.010,-1,-1.0,0,1
4,1818,1,5,1818.012,-1,-1.0,0,1
...,...,...,...,...,...,...,...,...
9995,1845,5,14,1845.366,110,13.3,1,1
9996,1845,5,15,1845.368,157,15.9,1,1
9997,1845,5,16,1845.371,117,13.7,1,1
9998,1845,5,17,1845.374,-1,-1.0,0,1


In [29]:
# Data Preprocessing
dates, spots, inverter = pre.preprocess(data)

# Reverse the data
dates = dates[::-1]
spots = spots[::-1]

ymax: 435.0 ymin: 0.0
x.shape: (9993,)
y.shape: (9993,)
y values scaler: 217.5


In [30]:
if mean_type == "gaussian":
    weights = gaussian(M=mean_length, std=0.1, sym=True)
    weights /= np.sum(weights) # normalise the weights
    spots = pre.running_mean_helper(spots, weights)
elif mean_type == "uniform":
    weights = np.ones(mean_length)/mean_length
    spots = pre.running_mean_helper(spots, weights)
else:
    pass


In [31]:
X = spots
index = dates
x_slid, y_slid, idx_slid = pre.sliding_window_main(X, X, index)
x_train, y_train, idx_train, x_val, y_val, idx_val, x_test, y_test, idx_test = pre.data_splitting_main(x_slid, y_slid, idx_slid)

x_slid.shape: (9991, 2, 1)
y_slid.shape: (9991, 1)
x_train.shape:  (5984, 2, 1)
y_train.shape:  (5984, 1)
idx_train.shape:  (5984,)
x_val.shape:  (2000, 2, 1)
y_val.shape:  (2000, 1)
idx_val.shape:  (2000,)
x_test.shape:  (2000, 2, 1)
y_test.shape:  (2000, 1)
idx_test.shape:  (2000,)



In [32]:
net = network.create_network()

In [33]:
history = network.trainer(net, x_train, y_train, x_val, y_val, verbose=1)
var_train = np.var(y_train)
var_val = np.var(y_val)
print("Variance in y_train:", var_train)
print("Variance in y_val:", var_val)

layer_size: 1 - loss: 0.0146 - val_loss: 0.0172
layer_size: 2 - loss: 0.0151 - val_loss: 0.0141


KeyboardInterrupt: 

In [ ]:
plotter.plot_predictions(net, x_train, y_train, idx_train, x_val, y_val, idx_val)
plotter.plot_loss_vs_epoch(history, var_train, var_val)

In [ ]:
predictor = network.create_network(predictor=True)
predictor.set_weights(net.get_weights())

idx_step = np.average(np.diff(idx_slid))
x_start = x_train[:batch_size, :, :]
idx_start = idx_train[:batch_size]
idx_end = 1819.0
args = (
    predictor,
    x_start,
    idx_start,
    idx_end,
    idx_step
)
predictor.reset_states()
# net.reset_states()
npred = predictor.predict(x_train, batch_size=1)
rpred, idx_rpred = network.predict_from_self(*args)

In [ ]:
# plt.figure(figsize=(10,8))
plt.plot(idx_train, y_train, label="Actual Value", marker="+")
plt.plot(idx_rpred, rpred, label="Recursive Prediction", marker="o")
plt.plot(idx_train, npred, label="Normal Prediction", marker="x")

plt.xlabel("Date")
plt.ylabel("Normalised Sunspot Numbers")
plt.legend()